# Interactive M5 EDA

## 1 Introduction
Welcome to an extensive Exploratory Data Analysis for the 5th Makridakis forecasting competitions (M5)! This notebook will grow over the coming days and weeks into a deep dive of all the relevant aspects of this challenge. Here’s all you need to know to get started:

Some Background: the Makridakis competitions (or M-competitions), organised by forecasting expert Spyros Makridakis, aim to provide a better understanding and advancement of forecasting methodology by comparing the performance of different methods in solving a well-defined, real-world problem. The first M-competition was held in 1982. The forth competition (M4) ran in 2018 and featured “100,000 time series and 61 forecasting methods” (source in link). According to forecasting researcher and practitioner Rob Hyndman the M-competitions “have had an enormous influence on the field of forecasting. They focused attention on what models produced good forecasts, rather than on the mathematical properties of those models”. This empirical approach is very similar to Kaggle’s trade-mark way of having the best machine learning algorithms engage in intense competition on diverse datasets. M5 is the first M-competition to be held on Kaggle.

The goal: We have been challenged to predict sales data provided by the retail giant Walmart 28 days into the future. This competition will run in 2 tracks: In addition to forecasting the values themselves in the Forecasting competition, we are simultaneously tasked to estimate the uncertainty of our predictions in the Uncertainty Distribution competition. Both competitions will have the same 28 day forecast horizon.

The data: We are working with 42,840 hierarchical time series. The data were obtained in the 3 US states of California (CA), Texas (TX), and Wisconsin (WI). “Hierarchical” here means that data can be aggregated on different levels: item level, department level, product category level, and state level. The sales information reaches back from Jan 2011 to June 2016. In addition to the sales numbers, we are also given corresponding data on prices, promotions, and holidays. Note, that we have been warned that most of the time series contain zero values.

The data comprises 3049 individual products from 3 categories and 7 departments, sold in 10 stores in 3 states. The hierachical aggregation captures the combinations of these factors. For instance, we can create 1 time series for all sales, 3 time series for all sales per state, and so on. The largest category is sales of all individual 3049 products per 10 stores for 30490 time series.

The training data comes in the shape of 3 separate files:

sales_train.csv: this is our main training data. It has 1 column for each of the 1941 days from 2011-01-29 and 2016-05-22; not including the validation period of 28 days until 2016-06-19. It also includes the IDs for item, department, category, store, and state. The number of rows is 30490 for all combinations of 30490 items and 10 stores.

sell_prices.csv: the store and item IDs together with the sales price of the item as a weekly average.

calendar.csv: dates together with related features like day-of-the week, month, year, and an 3 binary flags for whether the stores in each state allowed purchases with SNAP food stamps at this date (1) or not (0).

The metrics:

The point forecast submission are being evaluated using the Root Mean Squared Scaled Error (RMSSE), which is derived from the Mean Absolute Scaled Error (MASE) that was designed to be scale invariant and symmetric. In a similar way to the MASE, the RMSSE is scale invariant and symmetric, and measures the prediction error (i.e. forecast - truth) relative to a “naive forecast” that simply assumes that step i = step i-1. In contrast to the MASE, here both prediction error and naive error are scaled to account for the goal of estimating average values in the presence of many zeros.

The uncertainy distributions are being evaluated using the Weighted Scaled Pinball Loss (WSPL). We are asked to provide the 50%, 67%, 95%, and 99% uncertainty intervals together with the forecasted median.

Both metrics are computed for each time series and then averaged accross all time series including weights. The weights are proportional to the sales volume of the item, in dollars, to give more importance to high selling products. Note, that the weights are based on the last 28 days of the training data, and that those dates will be adjusted for the ultimate evaluation data, as confirmed by the organisers.

Need some inspiration? Beyond this notebook, I recommend you to check out other recent Kaggle time series competitions such as Wiki Traffic Forecasting, Favorita Grocery Sales Forecasting, Recruit Restaurant, or ASHRAE and read the Kernels and top ranking write-ups. Also, note that a few years ago Kaggle ran two Walmart recruiting competitions with the goal to forecast sales: Recruiting 1 and Recruiting 2. Last but not least, there was a relevant Playground competition a few years ago.

Let’s get started!

## 2 Preparations

#### load libs

In [78]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth',100)

#### load data

In [77]:
train = pd.read_csv('data/m5-forecasting-accuracy/sales_train_validation.csv')

PRICE_DTYPES = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float32" }
CAL_DTYPES={"event_name_1": "category", "event_name_2": "category", "event_type_1": "category", 
        "event_type_2": "category", "weekday": "category", 'wm_yr_wk': 'int16', "wday": "int16",
        "month": "int16", "year": "int16", "snap_CA": "float32", 'snap_TX': 'float32', 'snap_WI': 'float32' }

numcols = [f"d_{day}" for day in range(1,1914)]
catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
SALE_DTYPES = {numcol:"float32" for numcol in numcols} 
SALE_DTYPES.update({col: "category" for col in catcols if col != "id"})

# 加载price数据
price_data = pd.read_csv('data/m5-forecasting-accuracy/sell_prices.csv',dtype=PRICE_DTYPES)
# 加载cal数据
cal_data = pd.read_csv('data/m5-forecasting-accuracy/calendar.csv',dtype=CAL_DTYPES)
# 加载sale数据
sale_data = pd.read_csv('data/m5-forecasting-accuracy/sales_train_validation.csv',dtype=SALE_DTYPES,usecols=catcols+numcols)

In [91]:
sale_data = pd.melt(sale_data,
        id_vars = catcols,
        value_vars = [col for col in sale_data.columns if col.startswith("d_")],
        var_name = "d",
        value_name = "sales")


ValueError: No objects to concatenate

In [95]:
sale_data = sale_data.merge(cal_data, on= "d", copy = False)
sale_data = sale_data.merge(price_data, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,CA_1,HOBBIES,CA,d_1,0.0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,CA_1,HOBBIES,CA,d_1,0.0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,CA_1,HOBBIES,CA,d_1,0.0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,CA_1,HOBBIES,CA,d_1,0.0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,CA_1,HOBBIES,CA,d_1,0.0
...,...,...,...,...,...,...,...,...
99995,FOODS_3_203_CA_3_validation,FOODS_3_203,FOODS_3,CA_3,FOODS,CA,d_4,3.0
99996,FOODS_3_204_CA_3_validation,FOODS_3_204,FOODS_3,CA_3,FOODS,CA,d_4,0.0
99997,FOODS_3_205_CA_3_validation,FOODS_3_205,FOODS_3,CA_3,FOODS,CA,d_4,0.0
99998,FOODS_3_206_CA_3_validation,FOODS_3_206,FOODS_3,CA_3,FOODS,CA,d_4,0.0


In [102]:
sale_data['lag_7'] = sale_data[["id","sales"]].groupby("id")["sales"].shift(7)

sale_data['rmean_7_7'] = sale_data[["id", 'lag_7']].groupby("id")['lag_7'].transform(lambda x : x.rolling(7).mean())

In [105]:
sale_data[sale_data['id']=='HOBBIES_1_001_CA_1_validation'].head(100)

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,lag_7,rmean_7_7
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,CA_1,HOBBIES,CA,d_1,0.0,NaN,NaN
30490,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,CA_1,HOBBIES,CA,d_2,0.0,NaN,NaN
60980,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,CA_1,HOBBIES,CA,d_3,0.0,NaN,NaN
91470,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,CA_1,HOBBIES,CA,d_4,0.0,NaN,NaN
121960,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,CA_1,HOBBIES,CA,d_5,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2896550,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,CA_1,HOBBIES,CA,d_96,0.0,0.0,0.0
2927040,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,CA_1,HOBBIES,CA,d_97,0.0,0.0,0.0
2957530,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,CA_1,HOBBIES,CA,d_98,0.0,0.0,0.0
2988020,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,CA_1,HOBBIES,CA,d_99,0.0,0.0,0.0


## 3 Quick Look: File structure and content

In [79]:
train.head(5)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [80]:
prices.head(5)

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [81]:
prices.count()

store_id      6841121
item_id       6841121
wm_yr_wk      6841121
sell_price    6841121
dtype: int64

In [82]:
calendar.head(5)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [83]:
calendar.count()

date            1969
wm_yr_wk        1969
weekday         1969
wday            1969
month           1969
year            1969
d               1969
event_name_1     162
event_type_1     162
event_name_2       5
event_type_2       5
snap_CA         1969
snap_TX         1969
snap_WI         1969
dtype: int64

In [84]:
sale_data.head(5)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,1.0,3.0,0.0,1.0,1.0,1.0,3.0,0.0,1.0,1.0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,2.0,1.0,2.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,4.0,1.0,0.0,1.0,3.0,7.0,2.0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,2.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,2.0,4.0


We find:

There is one column each for the IDs of item, department, category, store, and state; plus a general ID that is a combination of the other IDs plus a flag for validation.

The sales per date are encoded as columns starting with the prefix d_. Those are the number of units sold per day (not the total amount of dollars).

We already see that there are quite a lot of zero values.

This data set has too many columns and rows to display them all:

In [24]:
train.count()

id          30490
item_id     30490
dept_id     30490
cat_id      30490
store_id    30490
            ...  
d_1909      30490
d_1910      30490
d_1911      30490
d_1912      30490
d_1913      30490
Length: 1919, dtype: int64

In [37]:
mutate = train['id'].apply(lambda x: 'validation' if x.find('validation') else 'training')
mutate.value_counts()

validation    30490
Name: id, dtype: int64

## 4 Visual Overview: Interactive time series plots

We will start our visual exploration by investigating a number of time series plots on different aggregation levels. Here is a short helper function to transform our wide data into a long format with a dates column in date format:

### 4.1 Individual item-level time series - random sample
Here we will sample 50 random time series from our training sample to get an idea about the data.

In the following plot, you can select the id of a time series (which is a concatenation of store and department) to display only the graphs that you’re interested in.

Currently, I don’t see a way to avoid having all time series plotted at the beginning. Select 1 from the input field to begin:

In [26]:
from datetime import datetime, date, timedelta
import matplotlib.dates as mdates

num_plots = 20
colormap = plt.cm.gist_ncar
min_date = datetime.strptime("2011-01-29", '%Y-%m-%d')
min_date + timedelta(days=10)

datetime.datetime(2011, 2, 8, 0, 0)

In [39]:
sample50_train = train.sample(n=50)
sample50_train.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
5405,FOODS_3_132_CA_2_validation,FOODS_3_132,FOODS_3,FOODS,CA_2,CA,6,3,1,2,...,1,2,0,3,0,1,0,1,1,1
8171,FOODS_2_247_CA_3_validation,FOODS_2_247,FOODS_2,FOODS,CA_3,CA,5,4,12,4,...,1,2,4,4,0,4,0,1,5,0
30190,FOODS_3_525_WI_3_validation,FOODS_3_525,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,10,7,6,10,3,2,3,5,7,6
21906,HOBBIES_2_148_WI_1_validation,HOBBIES_2_148,HOBBIES_2,HOBBIES,WI_1,WI,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
14637,FOODS_3_217_TX_1_validation,FOODS_3_217,FOODS_3,FOODS,TX_1,TX,1,0,0,0,...,0,1,1,3,2,2,0,1,1,0


In [56]:
import ggplot
dates = [min_date + timedelta(days=int(x.replace('d_',''))) for x in sample50_train.columns if x.startswith('d_')]
for index, row in sample50_train.iterrows():
    sales = [row[x] for  x in sample50_train.columns if x.startswith('d_')]
#     print(sales)
    print(ggplot(sales, dates))
    break

AttributeError: module 'pandas' has no attribute 'tslib'

In [28]:
tmp = map(lambda s: min_date + timedelta(days=int(s.replace('d_',''))) if s.startswith('d_') else s, train.columns)

In [30]:
train.columns = tmp

In [31]:
train.columns

Index([               'id',           'item_id',           'dept_id',
                  'cat_id',          'store_id',          'state_id',
       2011-01-30 00:00:00, 2011-01-31 00:00:00, 2011-02-01 00:00:00,
       2011-02-02 00:00:00,
       ...
       2016-04-16 00:00:00, 2016-04-17 00:00:00, 2016-04-18 00:00:00,
       2016-04-19 00:00:00, 2016-04-20 00:00:00, 2016-04-21 00:00:00,
       2016-04-22 00:00:00, 2016-04-23 00:00:00, 2016-04-24 00:00:00,
       2016-04-25 00:00:00],
      dtype='object', length=1919)

In [69]:
PRICE_DTYPES = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float32" }
price_data = pd.read_csv('data/m5-forecasting-accuracy/sell_prices.csv',dtype=PRICE_DTYPES)

In [72]:
[f"d_{day}" for day in range(1900,1914)]

['d_1900',
 'd_1901',
 'd_1902',
 'd_1903',
 'd_1904',
 'd_1905',
 'd_1906',
 'd_1907',
 'd_1908',
 'd_1909',
 'd_1910',
 'd_1911',
 'd_1912',
 'd_1913']